<a href="https://colab.research.google.com/github/rj2663972/RAG-Application/blob/main/RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Dependecies**

In [8]:
!pip install langchain_community langchainhub chromadb langchain langchain_openai

**Import Open AI API Keys**

In [10]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('open-api-key')

**Using WebBasedLoaders to Scrap a website and prepare documents**

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

**Creating chunks of the documents**

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
print(splits[0])
print(splits[1])
print(len(splits))

**Prepare Embeddings of the chunks and store them in Chroma DB Vector Store**

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(splits, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

**Create Prompt and LLM**

In [12]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [14]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

**Final Rag Pipeline**

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

**Test the Rag pipeline**

In [ ]:
rag_chain.invoke("Are the recordings of the course available? For how long?")